In [45]:
import pandas as pd

df = pd.read_csv("Categories_FikseHub.csv")
print(df.head())


  Type of Repairer Type of category Type of garment in category   Service  \
0           Tailor          Clothes                         Top      Hole   
1           Tailor          Clothes                         Top   Take in   
2           Tailor          Clothes                         Top   Take in   
3           Tailor          Clothes                         Top  Take out   
4           Tailor          Clothes                         Top  Take out   

            Description  Price  Estimated time in hours  
0        Price per hole  199.0                      NaN  
1  Only existing fabric  349.0                      NaN  
2     Additional fabric  549.0                      NaN  
3  Only existing fabric  349.0                      NaN  
4     Additional fabric  549.0                      NaN  


In [46]:
from datasets import load_dataset

categories_dataset = load_dataset("csv",data_files="Categories_FikseHub.csv",split="train")

print(categories_dataset)



Dataset({
    features: ['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours'],
    num_rows: 257
})


In [47]:
columns = categories_dataset.column_names
print(columns)

['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours']


In [48]:
categories_dataset.set_format("pandas")
df = categories_dataset[:]

In [49]:
print(df.head())

  Type of Repairer Type of category Type of garment in category   Service  \
0           Tailor          Clothes                         Top      Hole   
1           Tailor          Clothes                         Top   Take in   
2           Tailor          Clothes                         Top   Take in   
3           Tailor          Clothes                         Top  Take out   
4           Tailor          Clothes                         Top  Take out   

            Description  Price  Estimated time in hours  
0        Price per hole  199.0                      NaN  
1  Only existing fabric  349.0                      NaN  
2     Additional fabric  549.0                      NaN  
3  Only existing fabric  349.0                      NaN  
4     Additional fabric  549.0                      NaN  


In [50]:
print(categories_dataset.column_names) 

['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours']


In [51]:
def no_nulls(example):
    # Replace with your column names
    return all(example[col] is not None for col in example.keys())

categories_dataset = categories_dataset.filter(no_nulls)

print(categories_dataset)

Dataset({
    features: ['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours'],
    num_rows: 251
})


In [53]:
def concatenate_text(examples):
    # Create the new text field
    text = (
        str(examples["Type of Repairer"]) + "\n"
        + str(examples["Type of category"]) + "\n"
        + str(examples["Type of garment in category"]) + "\n"
        + str(examples["Service"]) + "\n"
        + str(examples["Description"]) + "\n"
        + str(examples["Price"]) + "\n"
        + str(examples["Estimated time in hours"])
    )
    # Add the new text field to the example
    examples["text"] = text
    return examples

categories_dataset = categories_dataset.map(concatenate_text)


Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [54]:
print(categories_dataset.column_names)  


['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours', 'text']


In [56]:
from transformers import AutoTokenizer, AutoModel

model_checkpoint= "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

In [57]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [58]:
#embedding = get_embeddings(categories_dataset["text"][0])

embedding = get_embeddings([categories_dataset["text"][0]])
embedding.shape

torch.Size([1, 768])

In [59]:
embeddings_dataset = categories_dataset.map(
    lambda x: {"embeddings": get_embeddings([str(x["text"])]).detach().cpu().numpy()[0]},
    remove_columns=[]
)


Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [76]:
print(categories_dataset.column_names)  


['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours', 'text']


In [60]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['embeddings'],
    num_rows: 251
})

In [70]:
print(embeddings_dataset.column_names) 

['embeddings']


In [74]:
import pandas as pd
def search_categories(query, dataset, k=5):
    
    query_embedding = get_embeddings([query]).detach().cpu().numpy()
    
    scores, samples = dataset.get_nearest_examples("embeddings", query_embedding, k=k)
    
    df = pd.DataFrame.from_dict(samples).assign(scores=scores)
    df["scores"] = scores
    columns_to_show = [
        "Type of Repairer",
        "Type of category",
        "Type of garment in category",
        "Service",
        "Description",
        "Price",
        "Estimated time in hours",
        "scores"
    ]
    
    # Return the sorted results
    return df[columns_to_show].sort_values("scores", ascending=False)



In [75]:
# Usage example:
results_df = search_categories("cotton shirt 199", embeddings_dataset)
print(results_df)

KeyError: "['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours'] not in index"

In [73]:

import pandas as pd
def search_categories(query, dataset, k=5):
    query_embedding = get_embeddings([query]).detach().cpu().numpy()
    scores, samples = dataset.get_nearest_examples("embeddings", query_embedding, k=k)
    df = pd.DataFrame.from_dict(samples).assign(scores=scores)
    df["scores"] = scores
    # Show all columns, including 'scores'
    return df.sort_values("scores", ascending=False)

# Usage example:
results_df = search_categories("cotton shirt 199", embeddings_dataset)
print(results_df)


                                          embeddings     scores
0  [-0.033357613, -0.4306201, -0.2655698, 0.14593...  46.653767
1  [-0.033357613, -0.4306201, -0.2655698, 0.14593...  46.653767
2  [-0.033357613, -0.4306201, -0.2655698, 0.14593...  46.653767
3  [-0.033357613, -0.4306201, -0.2655698, 0.14593...  46.653767
4  [-0.033357613, -0.4306201, -0.2655698, 0.14593...  46.653767


In [66]:
print("Available columns:", df.columns.tolist())


Available columns: ['Type of Repairer', 'Type of category', 'Type of garment in category', 'Service', 'Description', 'Price', 'Estimated time in hours']
